In [11]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import stumpy
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import numpy.testing as npt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
import plotly.express as px
import os
from vnstock import * #import all functions, including functions that provide OHLC data for charting
# from vnstock.chart import * # import chart functions

In [13]:
df = pd.read_csv('data/BaoCaoLaiLo_058C647873.csv', encoding='latin1', skiprows=5)

# Step 3: Rename columns according to the specified order
column_order = ['date', 'sell_vol', 'sell_price', 'sell_value', 'capital_price', 'capital_value', 'dividend', 'win_loss_value', 'win_loss_percent']
df.columns = column_order
df = df.drop(df.index[-1])

# Step 4: Display the resulting dataframe
print(df.head())

         date sell_vol sell_price  sell_value capital_price capital_value  \
0         BCG    1,200      9,780  11,736,000        10,150    12,180,000   
1  23/06/2023      600      9,950   5,970,000        10,150     6,090,000   
2  26/06/2023      600      9,610   5,766,000        10,150     6,090,000   
3         BMP      460     77,333  35,573,400        79,068    36,371,499   
4  30/05/2023        5     78,800     394,000        78,390       391,952   

  dividend win_loss_value win_loss_percent  
0        0       -444,000           -3.65%  
1      NaN       -120,000           -1.97%  
2      NaN       -324,000           -5.32%  
3        0       -798,099           -2.19%  
4      NaN          2,048            0.52%  


In [14]:
df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,dividend,win_loss_value,win_loss_percent
0,BCG,"1,200","9,780","11,736,000","10,150","12,180,000",0,"-444,000",-3.65%
1,23/06/2023,600,"9,950","5,970,000","10,150","6,090,000",NaN,"-120,000",-1.97%
2,26/06/2023,600,"9,610","5,766,000","10,150","6,090,000",NaN,"-324,000",-5.32%
3,BMP,460,"77,333","35,573,400","79,068","36,371,499",0,"-798,099",-2.19%
4,30/05/2023,5,"78,800","394,000","78,390","391,952",NaN,"2,048",0.52%


In [15]:
def preprocess_percent_sign(df, col ='win_loss_percent'):
    df[col] = df[col].str.rstrip('%')
    df[col] = df[col].astype('float32')
    return df

In [16]:
# Step 4: Remove the '%' symbol from the 'win_loss_percent' column
df = preprocess_percent_sign(df)

In [17]:
df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,dividend,win_loss_value,win_loss_percent
0,BCG,"1,200","9,780","11,736,000","10,150","12,180,000",0,"-444,000",-3.65
1,23/06/2023,600,"9,950","5,970,000","10,150","6,090,000",NaN,"-120,000",-1.97
2,26/06/2023,600,"9,610","5,766,000","10,150","6,090,000",NaN,"-324,000",-5.32
3,BMP,460,"77,333","35,573,400","79,068","36,371,499",0,"-798,099",-2.19
4,30/05/2023,5,"78,800","394,000","78,390","391,952",NaN,"2,048",0.52


In [18]:
def create_stock_nane_from_date(df):
    df['stock_name'] = df['date'].where(df['date'].str.len() == 3)
    df['stock_name'] = df['stock_name'].ffill().astype('category')
    df = df[df['date'].str.len() != 3]
    return df
df = create_stock_nane_from_date(df)

In [19]:
def convert_type(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    df = df.drop('dividend', axis=1)
    return df
df = convert_type(df)

In [20]:
num_cols = ['sell_vol', 'sell_price', 'sell_value', 'capital_price',
       'capital_value', 'win_loss_value', 'win_loss_percent']
df[num_cols] = df[num_cols].replace(',', '', regex=True)

# Convert columns to numeric
df[num_cols] = df[num_cols].apply(pd.to_numeric)
for col in num_cols:
    df[col] = df[col].astype(float)

In [26]:
df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,win_loss_value,win_loss_percent,stock_name
1,2023-06-23,600.0,9950.0,5970000.0,10150.0,6090000.0,-120000.0,-1.97,BCG
2,2023-06-26,600.0,9610.0,5766000.0,10150.0,6090000.0,-324000.0,-5.32,BCG
4,2023-05-30,5.0,78800.0,394000.0,78390.0,391952.0,2048.0,0.52,BMP
5,2023-05-31,100.0,80000.0,8000000.0,78390.0,7839038.0,160962.0,2.05,BMP
6,2023-06-02,100.0,77000.0,7700000.0,79269.0,7926904.0,-226904.0,-2.86,BMP


In [77]:
def my_strategy_analyse(df):
    """
    Analyze trading strategy performance metrics.

    Parameters:
    - df (pd.DataFrame): DataFrame containing trading data.

    Returns:
    - dict: Dictionary containing calculated metrics.
    """

    # Calculate win ratio
    total_trades = len(df)
    winning_trades = len(df[df['win_loss_value'] > 0])
    win_ratio = winning_trades / total_trades

    # Calculate payoff ratio
    average_win = df[df['win_loss_value'] > 0]['win_loss_value'].mean()
    average_loss = df[df['win_loss_value'] < 0]['win_loss_value'].mean()
    payoff_ratio = abs(average_win / average_loss)

    # Calculate largest winning trade
    largest_winning_trade = df['win_loss_value'].max()

    # Calculate largest losing trade
    largest_losing_trade = df['win_loss_value'].min()

    # Calculate largest winning trade and losing trade percentage 
    largest_winning_trade_percent = df['win_loss_percent'].max()
    largest_losing_trade_percent = df['win_loss_percent'].min()

    # Calculate average winning trade
    average_winning_trade = df[df['win_loss_value'] > 0]['win_loss_value'].mean()

    # Calculate average losing trade
    average_losing_trade = df[df['win_loss_value'] < 0]['win_loss_value'].mean()

    # Calculate largest % drawdown
    cumulative_returns = (1 + df['win_loss_percent'] / 100).cumprod()
    largest_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min()

    # Calculate average % drawdown
    average_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).mean()

    # Step 11: Calculate total days
    total_days = (df['date'].max() - df['date'].min()).days

    # Step 12: Calculate total win/loss percentage and value through time

    total_sell_value = df['sell_value'].sum()
    total_capital = df['capital_value'].sum()
    delta_sell_capital = total_sell_value - total_capital
    delta_sell_capital_percent = delta_sell_capital/total_capital

    df['year_month'] = df['date'].dt.to_period('M')
    average_trading_frequency = df.groupby('year_month').size().mean()

    stock_fee = ((0.088+0.1)/100) * df['capital_value'].sum()
    # Create a dictionary to store the calculated metrics
    metrics = {
        'Win Ratio': win_ratio,
        'Payoff Ratio': payoff_ratio,
        'Largest Winning Trade': largest_winning_trade,
        'Largest Losing Trade': largest_losing_trade,
        'Largest Winning Trade percent': largest_winning_trade_percent,
        'Largest Losing Trade percent': largest_losing_trade_percent,
        'Average Winning Trade': average_winning_trade,
        'Average Losing Trade': average_losing_trade,
        'Largest % Drawdown': largest_drawdown * 100,  # Convert to percentage
        'Average % Drawdown': average_drawdown * 100,  # Convert to percentage
        'Total Days': total_days,
        'total_sell_value': total_sell_value,
        'total_capital': total_capital,
        'delta_sell_capital': delta_sell_capital,
        'delta_sell_capital_percent': delta_sell_capital_percent*100,
        'Average Trading Frequency per Month': average_trading_frequency,
        'stock_fee': stock_fee,
        }

    return metrics


In [78]:
metrics = my_strategy_analyse(df)

In [79]:
metrics

{'Win Ratio': 0.6145038167938931,
 'Payoff Ratio': 1.3901079930969387,
 'Largest Winning Trade': 2057498.0,
 'Largest Losing Trade': -1906250.0,
 'Largest Winning Trade percent': 24.8700008392334,
 'Largest Losing Trade percent': -16.940000534057617,
 'Average Winning Trade': 330680.5714285714,
 'Average Losing Trade': -237881.21,
 'Largest % Drawdown': -57.65596273317476,
 'Average % Drawdown': -17.982174851701384,
 'Total Days': 180,
 'total_sell_value': 2076951650.0,
 'total_capital': 2047500200.0,
 'delta_sell_capital': 29451450.0,
 'delta_sell_capital_percent': 1.4384101159062157,
 'Average Trading Frequency per Month': 37.42857142857143,
 'stock_fee': 3849300.3759999997}

In [80]:
def plot_return(df, col = 'win_loss_value', figsize=(800, 600)):
    """
    Plot Daily Aggregated Sell Value and Capital Value using Plotly.

    Parameters:
    - df (pd.DataFrame): DataFrame containing trading data.
    - figsize (tuple, optional): Size of the plot (width, height). Default is (800, 600).

    Returns:
    - None: Displays the Plotly interactive chart.
    """
    # Step 1: Ensure the 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Step 2: Group by date and sum the values for each day
    daily_aggregated = df.groupby('date').agg({col: 'sum'}).reset_index()

    # Step 3: Plot the Sell Value and Capital Value using Plotly
    fig = px.line(daily_aggregated, x='date', y=[col],
                  title=f'Daily Aggregated {col}',
                  labels={'value': 'Value', 'date': 'Date'},
                  line_shape='linear', render_mode='svg')

    # Step 4: Customize the layout if needed
    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Value',
        showlegend=True,
    )

    # Step 5: Set the figsize
    fig.update_layout(width=figsize[0], height=figsize[1])

    # Step 6: Show the plot
    fig.show()

# Example Usage:
# Equity_Curve(your_dataframe, figsize=(1000, 800))


In [81]:
plot_return(df, figsize=(1000, 600))